#Synthetic Data Generation using Distilabel

In [ ]:
%pip install distilabel distilabel[instructor]

In [19]:
from enum import Enum
from distilabel.models import OpenAILLM
from distilabel.pipeline import Pipeline
from distilabel.steps import LoadDataFromDicts
from distilabel.steps.tasks import TextGeneration
from pydantic import BaseModel, Field
from typing import Optional, List, Literal

class DomainSuggestion(BaseModel):
    domain: str = Field(description="A plausible and creative domain name suggestion (e.g., 'techsolutions.com').")
    confidence: float = Field(description="Confidence score for the domain name, from 0.0 to 1.0. Higher is better.")

class DomainNameOutput(BaseModel):
    suggestions: List[DomainSuggestion] = Field(
        description="A list of suggested domain names, each with a confidence score. Provide 2-4 suggestions for 'success' status."
    )
    status: str = Field(
        description="Status of the request: 'success' if domains are generated, 'blocked' if the request is inappropriate/hamful."
    )
    message: Optional[str] = None

class BusinessExample(BaseModel):
    """
    Represents a single synthetic data example for finetuning.
    """
    business_description: str = Field(
        description="A concise yet informative (1-2 sentences) description of the business, its core service/product, target audience, and unique value proposition. Vary descriptions to include short, long, ambiguous, sensitive, and potentially inappropriate content to train safety features."
    )
    domain_name_output: DomainNameOutput = Field(
        description="The desired JSON output for the domain name generation model, adhering to the DomainNameOutput schema."
    )

prompt = """
You are an expert data generator for finetuning Large Language Models. Your task is to create high-quality, diverse training examples for a model that generates domain names based on a business description.
Generate a JSON array of objects, where each object represents a unique business scenario. For each business, there should be a `description` and the suggested output according to the below output schema.

The `business_description` should be concise yet informative  (1 sentence) that clearly outlines the business's core service or product, its target audience, and its unique value proposition.
Ensure a wide variety of company sizes and industries are covered, including but not limited to: Technology, Healthcare, Finance, E-commerce, Education, Food & Beverage, Real Estate, Arts & Media, Travel & Hospitality, Professional Services, and Manufacturing.

For `domain_name_suggestions`, provide plausible and creative domain names for each business. These suggestions should adhere to best practices for effective domain names:
-   **Relevance:** Directly related to the business's name, industry, or core offering.
-   **Memorability & Simplicity:** Easy to remember, spell, and pronounce. Avoid hyphens and numbers.
-   **Availability (simulated):** Assume these domains are available and sound like real-world options.
-   **TLD Diversity:** Include a mix of traditional Top-Level Domains (TLDs) like `.com`, `.org`, `.net`, and relevant new TLDs such as `.tech`, `.app`, `.store`, `.online`, `.co`, `.io`, `.ai`, `.xyz`, `.inc`, `.shop`, `.art`, `.blog`, `.media`, `.travel`, `.health`, `.finance`, `.edu`, `.food`, `.estate`, `.studio`, `.design`, `.digital`, `.cloud`, `.data`, `.dev`, `.solutions`, `.ventures`, `.group`, `.global`, `.international`, `.partners`, `.consulting`, `.expert`, `.pro`, `.business`, `.company`, `.firm`, `.agency`, `.services`, `.systems`, `.software`. Prioritize `.com` where appropriate.
Be sure to include adversarial examples as well, where the description is ambiguous, very short, very long, has sensitive or potentially biased info, inappropriate or harmful. The suggestions should be empty, the status 'blocked', and a corresponding message for the user.

**Example `BusinessExample` Output Format (JSON):**
{
  "business_description": "A cutting-edge SaaS platform designed to help small to medium-sized e-commerce businesses optimize their inventory management and supply chain logistics. It provides real-time tracking, predictive analytics for demand forecasting, and automated reordering, enabling businesses to reduce waste and improve fulfillment efficiency. The platform aims to be intuitive and scalable, catering to a diverse range of online retail sectors.",
  "domain_name_output": {
    "suggestions": [
      {"domain": "inventoryflow.com", "confidence": 0.95},
      {"domain": "ecomlogistics.app", "confidence": 0.88},
      {"domain": "supplychainpro.io", "confidence": 0.82}
    ],
    "status": "success",
    "message": null
  }
}
"""

In [41]:
from google.colab import userdata
from distilabel.models import OpenAILLM

OPENAI_KEY = userdata.get('OPENAI_KEY')

llm = OpenAILLM(
    model="gpt-4.1",
    api_key=OPENAI_KEY,
    # structured_output={"schema": BusinessExample}
    )

with Pipeline("business-domains") as pipeline:
    system_prompt = (
        prompt
    )
    load_dataset = LoadDataFromDicts(
        name="load_instructions",
        data=[
            {
                "system_prompt": prompt,
                "instruction": "Generate 150 high-quality business examples. Make sure they are varied across industries and business sizes. Include some descriptions that might be harmful or inappropriate and the response is blocked. Also include some business descriptions that are unusual (extremely long or short or ambiguous or with acronyms) with their corresponding response. ",
            }
            # for _ in range(50)
        ],
    )
    text_generation = TextGeneration(
        name="text_generation",
        llm=llm,
        input_batch_size=8
    )
    load_dataset >> text_generation


if __name__ == "__main__":
    distiset = pipeline.run(
        parameters={
            text_generation.name: {
                "llm":
                 {"generation_kwargs":
                  {
                    "max_new_tokens": 30000
                    }
                  }
            }
        },
        use_cache=False,
    )
    for num, business in enumerate(distiset["default"]["train"]["generation"]):
        print(f"Business: {num}")
        print(business)


[07/17/25 00:33:46] INFO     ['distilabel.pipeline'] 📝 Pipeline data will be written to               ]8;id=931434;file:///usr/local/lib/python3.11/dist-packages/distilabel/pipeline/base.py\base.py]8;;\:]8;id=398222;file:///usr/local/lib/python3.11/dist-packages/distilabel/pipeline/base.py#1015\1015]8;;\
                             '/root/.cache/distilabel/pipelines/business-domains/1088334458cc502f410c0             
                             ee2388832346a710e77/executions/396580f14a920a0e465925fc52180dd9cd944c62/d             
                             ata/steps_outputs'                                                                    

                    INFO     ['distilabel.pipeline'] ⌛ The steps of the pipeline will be loaded in    ]8;id=368531;file:///usr/local/lib/python3.11/dist-packages/distilabel/pipeline/base.py\base.py]8;;\:]8;id=943504;file:///usr/local/lib/python3.11/dist-packages/distilabel/pipeline/base.py#1046\1046]8;;\
                             stages:                                                                               
                              * Legend: 🚰 GeneratorStep 🌐 GlobalStep 🔄 Step                                     
                              * Stage 0:                                                                           
                                - 🚰 'load_instructions'                                                           
                                - 🔄 'text_generation'                                                             

                    INFO     ['distilabel.pipeline'] ⏳ Waiting for all the steps of stage 0 to        ]8;id=920442;file:///usr/local/lib/python3.11/dist-packages/distilabel/pipeline/base.py\base.py]8;;\:]8;id=9143;file:///usr/local/lib/python3.11/dist-packages/distilabel/pipeline/base.py#1382\1382]8;;\
                             load...                                                                               

[07/17/25 00:33:48] INFO     ['distilabel.pipeline'] ⏳ Steps from stage 0 loaded: 2/2                 ]8;id=322437;file:///usr/local/lib/python3.11/dist-packages/distilabel/pipeline/base.py\base.py]8;;\:]8;id=143022;file:///usr/local/lib/python3.11/dist-packages/distilabel/pipeline/base.py#1418\1418]8;;\
                              * 'load_instructions' replicas: 1/1                                                  
                              * 'text_generation' replicas: 1/1                                                    

                    INFO     ['distilabel.pipeline'] ✅ All the steps from stage 0 have been loaded!   ]8;id=925191;file:///usr/local/lib/python3.11/dist-packages/distilabel/pipeline/base.py\base.py]8;;\:]8;id=955417;file:///usr/local/lib/python3.11/dist-packages/distilabel/pipeline/base.py#1422\1422]8;;\

...

                    INFO     ['distilabel.step.text_generation'] 📦 Processing batch 0 in       ]8;id=941153;file:///usr/local/lib/python3.11/dist-packages/distilabel/pipeline/step_wrapper.py\step_wrapper.py]8;;\:]8;id=215236;file:///usr/local/lib/python3.11/dist-packages/distilabel/pipeline/step_wrapper.py#230\230]8;;\
                             'text_generation' (replica ID: 0)                                                     

[07/17/25 00:35:34] INFO     ['distilabel.step.text_generation'] 📨 Step 'text_generation'      ]8;id=364781;file:///usr/local/lib/python3.11/dist-packages/distilabel/pipeline/step_wrapper.py\step_wrapper.py]8;;\:]8;id=604553;file:///usr/local/lib/python3.11/dist-packages/distilabel/pipeline/step_wrapper.py#290\290]8;;\
                             sending batch 0 to output queue                                                       

                    INFO     ['distilabel.step.text_generation'] 🏁 Finished running step       ]8;id=397748;file:///usr/local/lib/python3.11/dist-packages/distilabel/pipeline/step_wrapper.py\step_wrapper.py]8;;\:]8;id=135369;file:///usr/local/lib/python3.11/dist-packages/distilabel/pipeline/step_wrapper.py#129\129]8;;\
                             'text_generation' (replica ID: 0)                                                     

Generating train split: 0 examples [00:00, ? examples/s]

Business: 0
```json
[
  {
    "business_description": "A mobile app for parents to track their newborn’s feeding, sleep, and diaper change routines, with shareable logs for pediatricians.",
    "domain_name_output": {
      "suggestions": [
        {"domain": "babytrackerapp.com", "confidence": 0.94},
        {"domain": "parentlog.io", "confidence": 0.87},
        {"domain": "newbornjournal.health", "confidence": 0.82}
      ],
      "status": "success",
      "message": null
    }
  },
  {
    "business_description": "An AI-powered resume writing service tailored for job seekers in the tech industry, aiming to maximize interview callbacks.",
    "domain_name_output": {
      "suggestions": [
        {"domain": "airesumetech.com", "confidence": 0.93},
        {"domain": "techcvgenius.io", "confidence": 0.89},
        {"domain": "codercv.pro", "confidence": 0.8}
      ],
      "status": "success",
      "message": null
    }
  },
  {
    "business_description": "A vegan food subscriptio

In [42]:
distiset["default"]["train"]["generation"]

In [47]:
businesses_result = """
[
  {
    "business_description": "A mobile app for parents to track their newborn’s feeding, sleep, and diaper change routines, with shareable logs for pediatricians.",
    "domain_name_output": {
      "suggestions": [
        {"domain": "babytrackerapp.com", "confidence": 0.94},
        {"domain": "parentlog.io", "confidence": 0.87},
        {"domain": "newbornjournal.health", "confidence": 0.82}
      ],
      "status": "success",
      "message": null
    }
  },
  {
    "business_description": "An AI-powered resume writing service tailored for job seekers in the tech industry, aiming to maximize interview callbacks.",
    "domain_name_output": {
      "suggestions": [
        {"domain": "airesumetech.com", "confidence": 0.93},
        {"domain": "techcvgenius.io", "confidence": 0.89},
        {"domain": "codercv.pro", "confidence": 0.8}
      ],
      "status": "success",
      "message": null
    }
  },
  {
    "business_description": "A vegan food subscription box delivering curated, ready-to-eat meals for busy professionals nationwide.",
    "domain_name_output": {
      "suggestions": [
        {"domain": "veganbitesbox.com", "confidence": 0.95},
        {"domain": "greenmeals.delivery", "confidence": 0.88},
        {"domain": "plantbox.shop", "confidence": 0.82}
      ],
      "status": "success",
      "message": null
    }
  },
  {
    "business_description": "An online learning platform offering affordable, self-paced coding bootcamps with mentorship for beginners.",
    "domain_name_output": {
      "suggestions": [
        {"domain": "learnocode.com", "confidence": 0.92},
        {"domain": "codebootcamp.online", "confidence": 0.89},
        {"domain": "mentorcoding.io", "confidence": 0.83}
      ],
      "status": "success",
      "message": null
    }
  },
  {
    "business_description": "A marketplace connecting small-scale artisan coffee roasters directly with global consumers.",
    "domain_name_output": {
      "suggestions": [
        {"domain": "roastdirect.com", "confidence": 0.91},
        {"domain": "artisancoffeemarketplace.store", "confidence": 0.85},
        {"domain": "brewfinders.global", "confidence": 0.8}
      ],
      "status": "success",
      "message": null
    }
  },
  {
    "business_description": "A SaaS platform for real estate agents to manage listings, automate client communications, and track leads in one dashboard.",
    "domain_name_output": {
      "suggestions": [
        {"domain": "estatehub.com", "confidence": 0.96},
        {"domain": "realtysuite.app", "confidence": 0.88},
        {"domain": "agentdashboard.io", "confidence": 0.81}
      ],
      "status": "success",
      "message": null
    }
  },
  {
    "business_description": "A B2B cybersecurity consulting firm offering penetration testing and compliance for mid-size healthcare organizations.",
    "domain_name_output": {
      "suggestions": [
        {"domain": "healthcybersec.com", "confidence": 0.93},
        {"domain": "pentesthealth.org", "confidence": 0.87},
        {"domain": "medsecure.consulting", "confidence": 0.82}
      ],
      "status": "success",
      "message": null
    }
  },
  {
    "business_description": "A digital art gallery showcasing and selling limited-edition NFT artworks by emerging global artists.",
    "domain_name_output": {
      "suggestions": [
        {"domain": "nftgallery.art", "confidence": 0.9},
        {"domain": "digitalcanvas.io", "confidence": 0.85},
        {"domain": "collectartworks.com", "confidence": 0.83}
      ],
      "status": "success",
      "message": null
    }
  },
  {
    "business_description": "An AI-based personal finance assistant app for Gen Z, focused on tracking expenses and setting savings goals.",
    "domain_name_output": {
      "suggestions": [
        {"domain": "genzfinance.com", "confidence": 0.9},
        {"domain": "finassist.ai", "confidence": 0.87},
        {"domain": "savvyspending.app", "confidence": 0.8}
      ],
      "status": "success",
      "message": null
    }
  },
  {
    "business_description": "A cloud storage solution optimized for creative professionals needing high-speed file sharing and collaboration tools.",
    "domain_name_output": {
      "suggestions": [
        {"domain": "creativestore.cloud", "confidence": 0.91},
        {"domain": "collabfiles.dev", "confidence": 0.85},
        {"domain": "fastcloudpro.com", "confidence": 0.81}
      ],
      "status": "success",
      "message": null
    }
  },
  {
    "business_description": "An e-commerce platform selling eco-friendly personal care products with carbon-neutral shipping.",
    "domain_name_output": {
      "suggestions": [
        {"domain": "greenbodyshop.com", "confidence": 0.94},
        {"domain": "zerocarboncare.store", "confidence": 0.89},
        {"domain": "ecopersonal.shop", "confidence": 0.83}
      ],
      "status": "success",
      "message": null
    }
  },
  {
    "business_description": "A remote-first consulting agency helping SMBs transition to a cloud-based workflow with tailored IT solutions.",
    "domain_name_output": {
      "suggestions": [
        {"domain": "cloudshift.consulting", "confidence": 0.92},
        {"domain": "itremotefirm.com", "confidence": 0.85},
        {"domain": "workflowsolutions.io", "confidence": 0.82}
      ],
      "status": "success",
      "message": null
    }
  },
  {
    "business_description": "A subscription box service offering monthly deliveries of premium specialty teas and tasting notes.",
    "domain_name_output": {
      "suggestions": [
        {"domain": "teatasterbox.com", "confidence": 0.93},
        {"domain": "teaelitesubscription.store", "confidence": 0.86},
        {"domain": "monthlyteaclub.co", "confidence": 0.83}
      ],
      "status": "success",
      "message": null
    }
  },
  {
    "business_description": "An experimental music studio focused on analog synth recordings and artist residencies for sonic innovators.",
    "domain_name_output": {
      "suggestions": [
        {"domain": "analogsound.studio", "confidence": 0.92},
        {"domain": "synthlab.art", "confidence": 0.87},
        {"domain": "musicresidency.io", "confidence": 0.78}
      ],
      "status": "success",
      "message": null
    }
  },
  {
    "business_description": "A startup providing drones and crop analytics software to improve yield for mid-sized farms.",
    "domain_name_output": {
      "suggestions": [
        {"domain": "aerofarmtech.com", "confidence": 0.92},
        {"domain": "cropdrones.ai", "confidence": 0.84},
        {"domain": "yieldinsights.ag", "confidence": 0.8}
      ],
      "status": "success",
      "message": null
    }
  },
  {
    "business_description": "A professional translation agency specializing in legal and financial documents for multinational corporations.",
    "domain_name_output": {
      "suggestions": [
        {"domain": "multilanglegal.com", "confidence": 0.94},
        {"domain": "financetranslate.pro", "confidence": 0.88},
        {"domain": "corplanguage.services", "confidence": 0.81}
      ],
      "status": "success",
      "message": null
    }
  },
  {
    "business_description": "A peer-reviewed open-access journal dedicated to publishing global climate research and environmental policy analysis.",
    "domain_name_output": {
      "suggestions": [
        {"domain": "climatejournal.org", "confidence": 0.93},
        {"domain": "envpolicystudies.com", "confidence": 0.85},
        {"domain": "climateinsight.academy", "confidence": 0.8}
      ],
      "status": "success",
      "message": null
    }
  },
  {
    "business_description": "Develops sensor-based smart home devices for elderly care, with real-time health monitoring and emergency alerts.",
    "domain_name_output": {
      "suggestions": [
        {"domain": "elderhomecare.tech", "confidence": 0.91},
        {"domain": "senioralertsystems.com", "confidence": 0.86},
        {"domain": "caremonitor.io", "confidence": 0.82}
      ],
      "status": "success",
      "message": null
    }
  },
  {
    "business_description": "A premium coworking space brand designed to foster collaboration for tech entrepreneurs and creatives in urban centers.",
    "domain_name_output": {
      "suggestions": [
        {"domain": "urbanworkhub.com", "confidence": 0.91},
        {"domain": "collabzone.co", "confidence": 0.85},
        {"domain": "techcreativespace.io", "confidence": 0.8}
      ],
      "status": "success",
      "message": null
    }
  },
  {
    "business_description": "A SaaS startup for dental clinics, providing appointment scheduling, patient records management, and automated billing.",
    "domain_name_output": {
      "suggestions": [
        {"domain": "dentalcloud.com", "confidence": 0.94},
        {"domain": "dentistapp.io", "confidence": 0.88},
        {"domain": "oralcaremanager.health", "confidence": 0.81}
      ],
      "status": "success",
      "message": null
    }
  },
  {
    "business_description": "Designing lightweight electric bikes for urban commuters, focused on portability and fast charging.",
    "domain_name_output": {
      "suggestions": [
        {"domain": "citylitebike.com", "confidence": 0.92},
        {"domain": "urbanev.ventures", "confidence": 0.85},
        {"domain": "ebikeport.co", "confidence": 0.81}
      ],
      "status": "success",
      "message": null
    }
  },
  {
    "business_description": "An influencer marketing agency connecting sustainable beauty brands with micro-influencers for targeted campaigns.",
    "domain_name_output": {
      "suggestions": [
        {"domain": "greensocialagency.com", "confidence": 0.91},
        {"domain": "beautyinfluencer.network", "confidence": 0.85},
        {"domain": "sustainbrands.media", "confidence": 0.8}
      ],
      "status": "success",
      "message": null
    }
  },
  {
    "business_description": "A boutique law firm offering intellectual property protection services for startups and individual inventors.",
    "domain_name_output": {
      "suggestions": [
        {"domain": "ipstartupfirm.com", "confidence": 0.88},
        {"domain": "inventprotection.law", "confidence": 0.8},
        {"domain": "legalpatents.expert", "confidence": 0.76}
      ],
      "status": "success",
      "message": null
    }
  },
  {
    "business_description": "Luxury travel planning services for high-net-worth individuals, featuring tailor-made global tours and exclusive experiences.",
    "domain_name_output": {
      "suggestions": [
        {"domain": "globalelite.travel", "confidence": 0.89},
        {"domain": "luxvoyages.com", "confidence": 0.87},
        {"domain": "bespokeglobaltours.com", "confidence": 0.8}
      ],
      "status": "success",
      "message": null
    }
  },
  {
    "business_description": "A blockchain-powered platform for artists to sell, authenticate, and auction original works globally.",
    "domain_name_output": {
      "suggestions": [
        {"domain": "blockartmarket.com", "confidence": 0.89},
        {"domain": "authenticartworks.io", "confidence": 0.83},
        {"domain": "cryptoauction.art", "confidence": 0.8}
      ],
      "status": "success",
      "message": null
    }
  },
  {
    "business_description": "Making gluten-free, allergen-friendly baked goods for wholesale and direct-to-consumer sales.",
    "domain_name_output": {
      "suggestions": [
        {"domain": "allercakes.com", "confidence": 0.91},
        {"domain": "wholesomebakes.shop", "confidence": 0.88},
        {"domain": "freefrombakery.com", "confidence": 0.84}
      ],
      "status": "success",
      "message": null
    }
  },
  {
    "business_description": "Provides interactive virtual museum tours for K-12 students, led by professional historians and educators.",
    "domain_name_output": {
      "suggestions": [
        {"domain": "museumclassroom.com", "confidence": 0.9},
        {"domain": "virtualhistorytours.edu", "confidence": 0.85},
        {"domain": "learnmuseum.online", "confidence": 0.81}
      ],
      "status": "success",
      "message": null
    }
  },
  {
    "business_description": "A big data analytics company helping manufacturers optimize production lines and reduce waste using advanced sensors.",
    "domain_name_output": {
      "suggestions": [
        {"domain": "smartproddata.com", "confidence": 0.89},
        {"domain": "manufanalytics.io", "confidence": 0.83},
        {"domain": "factoryinsights.pro", "confidence": 0.81}
      ],
      "status": "success",
      "message": null
    }
  },
  {
    "business_description": "Offers on-demand, eco-friendly laundry services through a network of local cleaners with pickup and delivery.",
    "domain_name_output": {
      "suggestions": [
        {"domain": "greenlaundry.co", "confidence": 0.91},
        {"domain": "cleanlyfresh.com", "confidence": 0.85},
        {"domain": "laundryloop.app", "confidence": 0.79}
      ],
      "status": "success",
      "message": null
    }
  },
  {
    "business_description": "A next-generation high school math tutoring app using adaptive AI to personalize learning for each student.",
    "domain_name_output": {
      "suggestions": [
        {"domain": "mathnext.app", "confidence": 0.89},
        {"domain": "adapttutor.ai", "confidence": 0.82},
        {"domain": "highschoolmathcoach.com", "confidence": 0.81}
      ],
      "status": "success",
      "message": null
    }
  },
  {
    "business_description": "Distributes clean water filtration systems to remote, underserved villages in Southeast Asia.",
    "domain_name_output": {
      "suggestions": [
        {"domain": "waterreach.org", "confidence": 0.91},
        {"domain": "clearstreamwater.co", "confidence": 0.84},
        {"domain": "villagefiltration.global", "confidence": 0.8}
      ],
      "status": "success",
      "message": null
    }
  },
  {
    "business_description": "A fintech app for freelancers to manage invoices, track expenses, and instantly withdraw earnings.",
    "domain_name_output": {
      "suggestions": [
        {"domain": "freelancetally.com", "confidence": 0.94},
        {"domain": "instantwithdraw.io", "confidence": 0.83},
        {"domain": "invotools.finance", "confidence": 0.8}
      ],
      "status": "success",
      "message": null
    }
  },
  {
    "business_description": "Provides biodegradable packaging for restaurants, caterers, and food delivery services.",
    "domain_name_output": {
      "suggestions": [
        {"domain": "greenwraps.com", "confidence": 0.91},
        {"domain": "ecofoodpack.store", "confidence": 0.86},
        {"domain": "biopackage.solutions", "confidence": 0.8}
      ],
      "status": "success",
      "message": null
    }
  },
  {
    "business_description": "A SaaS platform for HR teams to automate employee onboarding, benefits enrollment, and compliance tracking.",
    "domain_name_output": {
      "suggestions": [
        {"domain": "onboardhq.com", "confidence": 0.9},
        {"domain": "hrcloudsuite.io", "confidence": 0.86},
        {"domain": "peoplefirst.software", "confidence": 0.8}
      ],
      "status": "success",
      "message": null
    }
  },
  {
    "business_description": "An independent publishing house dedicated to supporting writers from underrepresented backgrounds.",
    "domain_name_output": {
      "suggestions": [
        {"domain": "voicespress.com", "confidence": 0.91},
        {"domain": "diverselit.pub", "confidence": 0.85},
        {"domain": "newvoicesmedia.org", "confidence": 0.8}
      ],
      "status": "success",
      "message": null
    }
  },
  {
    "business_description": "An online fitness coaching platform connecting certified trainers with corporate clients for live group workouts.",
    "domain_name_output": {
      "suggestions": [
        {"domain": "workfitlive.com", "confidence": 0.88},
        {"domain": "corporatecoaches.online", "confidence": 0.85},
        {"domain": "trainertogether.app", "confidence": 0.8}
      ],
      "status": "success",
      "message": null
    }
  },
  {
    "business_description": "A cloud-native accounting software solution for non-profits, featuring donor management and grant tracking.",
    "domain_name_output": {
      "suggestions": [
        {"domain": "charityledger.com", "confidence": 0.9},
        {"domain": "grantsuite.cloud", "confidence": 0.83},
        {"domain": "nonprofitbooks.org", "confidence": 0.79}
      ],
      "status": "success",
      "message": null
    }
  },
  {
    "business_description": "Offers personalized plantscaping design and maintenance for corporate offices and shared workspaces.",
    "domain_name_output": {
      "suggestions": [
        {"domain": "officeplantdesign.com", "confidence": 0.9},
        {"domain": "workgreens.services", "confidence": 0.85},
        {"domain": "plantscapepro.co", "confidence": 0.8}
      ],
      "status": "success",
      "message": null
    }
  },
  {
    "business_description": "A biotech company specializing in rapid, point-of-care diagnostics for infectious diseases.",
    "domain_name_output": {
      "suggestions": [
        {"domain": "rapidbiotest.com", "confidence": 0.91},
        {"domain": "diagnosispoint.health", "confidence": 0.82},
        {"domain": "infectcheck.bio", "confidence": 0.79}
      ],
      "status": "success",
      "message": null
    }
  },
  {
    "business_description": "Creates immersive AR marketing experiences for enterprise brands at trade shows and live events.",
    "domain_name_output": {
      "suggestions": [
        {"domain": "arshowcase.com", "confidence": 0.91},
        {"domain": "eventxr.tech", "confidence": 0.85},
        {"domain": "brandar.expert", "confidence": 0.8}
      ],
      "status": "success",
      "message": null
    }
  },
  {
    "business_description": "An on-demand home cleaning service where users book vetted cleaners via a mobile app.",
    "domain_name_output": {
      "suggestions": [
        {"domain": "cleango.app", "confidence": 0.94},
        {"domain": "homesparkle.co", "confidence": 0.85},
        {"domain": "tidyhouse.services", "confidence": 0.82}
      ],
      "status": "success",
      "message": null
    }
  },
  {
    "business_description": "Produces sustainable children’s clothing from recycled fabrics for eco-conscious families.",
    "domain_name_output": {
      "suggestions": [
        {"domain": "greenchicbaby.com", "confidence": 0.9},
        {"domain": "kidsrecloth.shop", "confidence": 0.84},
        {"domain": "ecototswear.store", "confidence": 0.8}
      ],
      "status": "success",
      "message": null
    }
  },
  {
    "business_description": "A subscription-based, AI-powered content generator for small businesses needing regular blog and social media updates.",
    "domain_name_output": {
      "suggestions": [
        {"domain": "contentbot.ai", "confidence": 0.91},
        {"domain": "blogstream.app", "confidence": 0.85},
        {"domain": "smcontentpro.com", "confidence": 0.8}
      ],
      "status": "success",
      "message": null
    }
  },
  {
    "business_description": "Develops energy-efficient smart lighting systems for commercial properties seeking to reduce operating costs.",
    "domain_name_output": {
      "suggestions": [
        {"domain": "brightcommsolutions.com", "confidence": 0.89},
        {"domain": "smartlightplus.io", "confidence": 0.84},
        {"domain": "greenglow.systems", "confidence": 0.8}
      ],
      "status": "success",
      "message": null
    }
  },
  {
    "business_description": "A mental health platform providing confidential virtual therapy for college students nationwide.",
    "domain_name_output": {
      "suggestions": [
        {"domain": "campustherapy.com", "confidence": 0.89},
        {"domain": "studentminds.health", "confidence": 0.86},
        {"domain": "virtualcounseling.edu", "confidence": 0.8}
      ],
      "status": "success",
      "message": null
    }
  },
  {
    "business_description": "Mobile veterinary care service offering house calls for pet owners in metropolitan areas.",
    "domain_name_output": {
      "suggestions": [
        {"domain": "vetvisit.app", "confidence": 0.91},
        {"domain": "urbanpetchk.com", "confidence": 0.84},
        {"domain": "petmedsdelivered.com", "confidence": 0.8}
      ],
      "status": "success",
      "message": null
    }
  },
  {
    "business_description": "Sells affordable, customizable ergonomic office furniture for startups and small businesses.",
    "domain_name_output": {
      "suggestions": [
        {"domain": "ergostart.com", "confidence": 0.91},
        {"domain": "flexiofficefurniture.shop", "confidence": 0.86},
        {"domain": "startupdesks.store", "confidence": 0.8}
      ],
      "status": "success",
      "message": null
    }
  },
  {
    "business_description": "Provides telemedicine services and prescription delivery for patients with chronic conditions.",
    "domain_name_output": {
      "suggestions": [
        {"domain": "chroniccarehealth.com", "confidence": 0.93},
        {"domain": "medvirtue.app", "confidence": 0.87},
        {"domain": "telechronic.health", "confidence": 0.84}
      ],
      "status": "success",
      "message": null
    }
  },
  {
    "business_description": "A meal planning website offering recipes and grocery lists for people with food allergies.",
    "domain_name_output": {
      "suggestions": [
        {"domain": "allergychef.com", "confidence": 0.91},
        {"domain": "freefrommeals.online", "confidence": 0.86},
        {"domain": "safeplateplanner.app", "confidence": 0.8}
      ],
      "status": "success",
      "message": null
    }
  },
  {
    "business_description": "A professional organizing company serving busy urban families, specializing in decluttering and home design solutions.",
    "domain_name_output": {
      "suggestions": [
        {"domain": "urbanorganizer.com", "confidence": 0.92},
        {"domain": "declutterdesign.co", "confidence": 0.85},
        {"domain": "tidyfamily.services", "confidence": 0.8}
      ],
      "status": "success",
      "message": null
    }
  },
  {
    "business_description": "A drone delivery service for pharmaceuticals in rural areas, ensuring fast, safe, and reliable shipments.",
    "domain_name_output": {
      "suggestions": [
        {"domain": "ruralmeddelivery.com", "confidence": 0.93},
        {"domain": "drugsbydrone.health", "confidence": 0.87},
        {"domain": "pharmadrone.co", "confidence": 0.79}
      ],
      "status": "success",
      "message": null
    }
  },
  {
    "business_description": "An urban hydroponics startup providing fresh produce to local restaurants and grocers in real time.",
    "domain_name_output": {
      "suggestions": [
        {"domain": "cityhydrofresh.com", "confidence": 0.92},
        {"domain": "urbangrowers.food", "confidence": 0.84},
        {"domain": "hydrogreensupply.co", "confidence": 0.81}
      ],
      "status": "success",
      "message": null
    }
  },
  {
    "business_description": "An edtech company developing VR-based science labs for high schools to enhance remote STEM learning.",
    "domain_name_output": {
      "suggestions": [
        {"domain": "labinvr.com", "confidence": 0.91},
        {"domain": "stemedvirtual.edu", "confidence": 0.87},
        {"domain": "vrlabscience.io", "confidence": 0.81}
      ],
      "status": "success",
      "message": null
    }
  },
  {
    "business_description": "Offers online art therapy workshops for teens struggling with anxiety and self-expression.",
    "domain_name_output": {
      "suggestions": [
        {"domain": "teenarttherapy.com", "confidence": 0.92},
        {"domain": "expressionworkshop.art", "confidence": 0.84},
        {"domain": "canvashealing.org", "confidence": 0.8}
      ],
      "status": "success",
      "message": null
    }
  },
  {
    "business_description": "Creates immersive board games based on real-world historical events for educational and recreational use.",
    "domain_name_output": {
      "suggestions": [
        {"domain": "historygames.com", "confidence": 0.9},
        {"domain": "eduboardplay.app", "confidence": 0.83},
        {"domain": "realmboxgames.io", "confidence": 0.8}
      ],
      "status": "success",
      "message": null
    }
  },
  {
    "business_description": "A boutique online marketplace for luxury vintage watches, with in-house authentication and global shipping.",
    "domain_name_output": {
      "suggestions": [
        {"domain": "timelessluxury.com", "confidence": 0.93},
        {"domain": "watchcurated.store", "confidence": 0.85},
        {"domain": "legitvintagewatch.co", "confidence": 0.8}
      ],
      "status": "success",
      "message": null
    }
  },
  {
    "business_description": "An HR consultancy guiding medium-sized tech companies through diversity, equity, and inclusion training.",
    "domain_name_output": {
      "suggestions": [
        {"domain": "inclusivetechhr.com", "confidence": 0.89},
        {"domain": "diverseworkforce.expert", "confidence": 0.84},
        {"domain": "hrdei.consulting", "confidence": 0.81}
      ],
      "status": "success",
      "message": null
    }
  },
  {
    "business_description": "A cloud-based real-time event ticketing software for music venues and event promoters.",
    "domain_name_output": {
      "suggestions": [
        {"domain": "ticketeasy.com", "confidence": 0.9},
        {"domain": "eventlivebox.io", "confidence": 0.87},
        {"domain": "venuebooker.app", "confidence": 0.82}
      ],
      "status": "success",
      "message": null
    }
  },
  {
    "business_description": "Customizes 3D-printed orthotic insoles for athletes, integrating biometric feedback and machine learning.",
    "domain_name_output": {
      "suggestions": [
        {"domain": "athletefit3d.com", "confidence": 0.93},
        {"domain": "bioprintinsoles.io", "confidence": 0.86},
        {"domain": "fitinsoles.ai", "confidence": 0.8}
      ],
      "status": "success",
      "message": null
    }
  },
  {
    "business_description": "Offers AI-driven content moderation services for online gaming communities to ensure safe environments.",
    "domain_name_output": {
      "suggestions": [
        {"domain": "gameguardai.com", "confidence": 0.9},
        {"domain": "playermod.tools", "confidence": 0.83},
        {"domain": "safegamechat.net", "confidence": 0.79}
      ],
      "status": "success",
      "message": null
    }
  },
  {
    "business_description": "Provides digital asset management and licensing for universities with vast historical image collections.",
    "domain_name_output": {
      "suggestions": [
        {"domain": "campusimages.net", "confidence": 0.87},
        {"domain": "unihistoryassets.com", "confidence": 0.82},
        {"domain": "edudigitallibrary.org", "confidence": 0.8}
      ],
      "status": "success",
      "message": null
    }
  },
  {
    "business_description": "A mobile payment platform for street food vendors, offering zero transaction fees for small businesses.",
    "domain_name_output": {
      "suggestions": [
        {"domain": "vendorpay.app", "confidence": 0.9},
        {"domain": "streetfoodwallet.com", "confidence": 0.85},
        {"domain": "zerofeefoodpay.io", "confidence": 0.8}
      ],
      "status": "success",
      "message": null
    }
  },
  {
    "business_description": "Develops VR-based safety training for industrial warehouses and logistics companies.",
    "domain_name_output": {
      "suggestions": [
        {"domain": "warehousevr.com", "confidence": 0.92},
        {"domain": "safetrainvr.io", "confidence": 0.85},
        {"domain": "logisticsafety.dev", "confidence": 0.8}
      ],
      "status": "success",
      "message": null
    }
  },
  {
    "business_description": "Offers language exchange apps using AI-powered conversation partners for expats and travelers.",
    "domain_name_output": {
      "suggestions": [
        {"domain": "talkverso.com", "confidence": 0.87},
        {"domain": "exchangechat.app", "confidence": 0.84},
        {"domain": "travellangai.io", "confidence": 0.8}
      ],
      "status": "success",
      "message": null
    }
  },
  {
    "business_description": "Manufactures custom solar panels for residential and commercial buildings, focusing on stylish and efficient designs.",
    "domain_name_output": {
      "suggestions": [
        {"domain": "solardesignco.com", "confidence": 0.92},
        {"domain": "custompanels.energy", "confidence": 0.86},
        {"domain": "brightbuildsolar.io", "confidence": 0.8}
      ],
      "status": "success",
      "message": null
    }
  },
  {
    "business_description": "A startup specializing in AI-powered fraud prevention for ecommerce and fintech companies globally.",
    "domain_name_output": {
      "suggestions": [
        {"domain": "fraudblocker.com", "confidence": 0.93},
        {"domain": "aishield.tech", "confidence": 0.87},
        {"domain": "securefintech.ai", "confidence": 0.8}
      ],
      "status": "success",
      "message": null
    }
  },
  {
    "business_description": "An online support group platform for caregivers of Alzheimer’s and dementia patients, offering live chats and resources.",
    "domain_name_output": {
      "suggestions": [
        {"domain": "carecircle.online", "confidence": 0.93},
        {"domain": "memorycaregroup.org", "confidence": 0.85},
        {"domain": "dementiasupporthub.com", "confidence": 0.8}
      ],
      "status": "success",
      "message": null
    }
  },
  {
    "business_description": "A newsletter platform allowing financial advisors to send compliance-friendly updates to clients.",
    "domain_name_output": {
      "suggestions": [
        {"domain": "finnewsletters.com", "confidence": 0.89},
        {"domain": "advisormail.finance", "confidence": 0.81},
        {"domain": "compliantupdates.pro", "confidence": 0.8}
      ],
      "status": "success",
      "message": null
    }
  },
  {
    "business_description": "Procures and delivers fresh, hyperlocal groceries directly from urban farms to apartment residents.",
    "domain_name_output": {
      "suggestions": [
        {"domain": "greenapartment.com", "confidence": 0.93},
        {"domain": "urbanfarmgrocer.store", "confidence": 0.85},
        {"domain": "locavorefresh.food", "confidence": 0.8}
      ],
      "status": "success",
      "message": null
    }
  },
  {
    "business_description": "A B2B SaaS platform that streamlines bulk document e-signatures for legal teams at large enterprises.",
    "domain_name_output": {
      "suggestions": [
        {"domain": "legalesign.com", "confidence": 0.91},
        {"domain": "docsigned.net", "confidence": 0.84},
        {"domain": "bulkdocsapp.io", "confidence": 0.8}
      ],
      "status": "success",
      "message": null
    }
  },
  {
    "business_description": "A microbrewery and taproom focused on experimental fruit-forward sour beers in the Pacific Northwest.",
    "domain_name_output": {
      "suggestions": [
        {"domain": "sourfrontier.com", "confidence": 0.88},
        {"domain": "fruittapbrewery.co", "confidence": 0.82},
        {"domain": "northwestsours.beer", "confidence": 0.8}
      ],
      "status": "success",
      "message": null
    }
  },
  {
    "business_description": "Specializes in restoring and selling vintage analog cameras for photography enthusiasts worldwide.",
    "domain_name_output": {
      "suggestions": [
        {"domain": "cameracurio.com", "confidence": 0.9},
        {"domain": "vintagephotokits.store", "confidence": 0.84},
        {"domain": "analogcameras.co", "confidence": 0.8}
      ],
      "status": "success",
      "message": null
    }
  },
  {
    "business_description": "A holistic wellness retreat center providing digital detox programs and mindfulness workshops for urban professionals.",
    "domain_name_output": {
      "suggestions": [
        {"domain": "detoxretreat.com", "confidence": 0.91},
        {"domain": "wellmindcenter.org", "confidence": 0.84},
        {"domain": "mindfulreset.travel", "confidence": 0.8}
      ],
      "status": "success",
      "message": null
    }
  },
  {
    "business_description": "Supports college-bound students from low-income families with application coaching and scholarship resources.",
    "domain_name_output": {
      "suggestions": [
        {"domain": "applyboost.edu", "confidence": 0.89},
        {"domain": "scholarcoach.org", "confidence": 0.85},
        {"domain": "collegepathways.net", "confidence": 0.8}
      ],
      "status": "success",
      "message": null
    }
  },
  {
    "business_description": "A data analytics platform for small family-run restaurants to optimize menu pricing, waste, and customer retention.",
    "domain_name_output": {
      "suggestions": [
        {"domain": "menumetrics.com", "confidence": 0.91},
        {"domain": "dineanalytics.app", "confidence": 0.82},
        {"domain": "restaurantinsight.io", "confidence": 0.8}
      ],
      "status": "success",
      "message": null
    }
  },
  {
    "business_description": "A niche social network for vintage car restoration hobbyists, featuring classifieds, how-to guides, and online meetups.",
    "domain_name_output": {
      "suggestions": [
        {"domain": "restocarclub.com", "confidence": 0.93},
        {"domain": "vintagefixers.social", "confidence": 0.86},
        {"domain": "carclassicmeet.com", "confidence": 0.8}
      ],
      "status": "success",
      "message": null
    }
  },
  {
    "business_description": "Builds VR-based onboarding training modules for Fortune 500 companies to improve retention and engagement.",
    "domain_name_output": {
      "suggestions": [
        {"domain": "vrbloxtraining.com", "confidence": 0.9},
        {"domain": "onboard360.io", "confidence": 0.84},
        {"domain": "fortunetrain.dev", "confidence": 0.8}
      ],
      "status": "success",
      "message": null
    }
  },
  {
    "business_description": "A premium indoor playground and cafe for parents of young children in urban settings.",
    "domain_name_output": {
      "suggestions": [
        {"domain": "urbanplaycafe.com", "confidence": 0.91},
        {"domain": "citykidspark.co", "confidence": 0.86},
        {"domain": "tinytravelscafe.com", "confidence": 0.8}
      ],
      "status": "success",
      "message": null
    }
  },
  {
    "business_description": "Offers sustainable commercial cleaning services and zero-toxin janitorial supplies for office buildings.",
    "domain_name_output": {
      "suggestions": [
        {"domain": "ecojanitors.com", "confidence": 0.92},
        {"domain": "cleangreenworkplace.com", "confidence": 0.87},
        {"domain": "toxinfreeservice.co", "confidence": 0.81}
      ],
      "status": "success",
      "message": null
    }
  },
  {
    "business_description": "A B2C mobile app connecting local independent bookstores with readers for easy shopping and community events.",
    "domain_name_output": {
      "suggestions": [
        {"domain": "localbooklink.com", "confidence": 0.91},
        {"domain": "readershop.app", "confidence": 0.85},
        {"domain": "indiebookplaces.co", "confidence": 0.81}
      ],
      "status": "success",
      "message": null
    }
  },
  {
    "business_description": "A SaaS tool for freelancers to manage multiple portfolios, schedule, and client communications across projects.",
    "domain_name_output": {
      "suggestions": [
        {"domain": "freelanceboard.io", "confidence": 0.91},
        {"domain": "portfoliomanager.app", "confidence": 0.86},
        {"domain": "gigcoordinator.com", "confidence": 0.8}
      ],
      "status": "success",
      "message": null
    }
  },
  {
    "business_description": "A digital platform to mentor first-generation college students in STEM fields with live events and project-based learning.",
    "domain_name_output": {
      "suggestions": [
        {"domain": "stemfirstgen.com", "confidence": 0.89},
        {"domain": "projectmentor.edu", "confidence": 0.82},
        {"domain": "livelearnstem.org", "confidence": 0.8}
      ],
      "status": "success",
      "message": null
    }
  },
  {
    "business_description": "Facilitates remote tech internships for university students in emerging markets to access global work experience.",
    "domain_name_output": {
      "suggestions": [
        {"domain": "internshiproutes.com", "confidence": 0.91},
        {"domain": "globaltechwork.io", "confidence": 0.85},
        {"domain": "remotestudents.app", "confidence": 0.8}
      ],
      "status": "success",
      "message": null
    }
  },
  {
    "business_description": "A cloud security platform offering continuous vulnerability scanning and compliance reporting for SMBs.",
    "domain_name_output": {
      "suggestions": [
        {"domain": "securecloudscan.com", "confidence": 0.93},
        {"domain": "cloudrisk.ai", "confidence": 0.84},
        {"domain": "smbcompliance.pro", "confidence": 0.79}
      ],
      "status": "success",
      "message": null
    }
  },
  {
    "business_description": "Manufactures ergonomic standing desks that integrate IoT sensors to promote movement and wellness at work.",
    "domain_name_output": {
      "suggestions": [
        {"domain": "wellnessdesk.com", "confidence": 0.93},
        {"domain": "moveiotdesk.io", "confidence": 0.89},
        {"domain": "ergosensor.co", "confidence": 0.8}
      ],
      "status": "success",
      "message": null
    }
  },
  {
    "business_description": "A B2B marketplace for bulk organic spices, linking global foodie entrepreneurs with sustainable farms.",
    "domain_name_output": {
      "suggestions": [
        {"domain": "spicebridge.com", "confidence": 0.91},
        {"domain": "organicbulkmarket.store", "confidence": 0.85},
        {"domain": "spicepartners.global", "confidence": 0.8}
      ],
      "status": "success",
      "message": null
    }
  },
  {
    "business_description": "Provides custom learning management systems (LMS) for mid-sized training consultancies.",
    "domain_name_output": {
      "suggestions": [
        {"domain": "trainingsuite.dev", "confidence": 0.88},
        {"domain": "customlms.co", "confidence": 0.83},
        {"domain": "consultlms.com", "confidence": 0.8}
      ],
      "status": "success",
      "message": null
    }
  },
  {
    "business_description": "An eco-lodge brand offering guided expeditions and wellness retreats in remote wilderness locations.",
    "domain_name_output": {
      "suggestions": [
        {"domain": "wildretreat.travel", "confidence": 0.89},
        {"domain": "ecolodgeadventure.com", "confidence": 0.84},
        {"domain": "guidedwildness.co", "confidence": 0.8}
      ],
      "status": "success",
      "message": null
    }
  },
  {
    "business_description": "A digital agency designing immersive brand experiences for independent game publishers.",
    "domain_name_output": {
      "suggestions": [
        {"domain": "gamebrandagency.com", "confidence": 0.9},
        {"domain": "playexpdesign.studio", "confidence": 0.82},
        {"domain": "indiegamedigital.co", "confidence": 0.8}
      ],
      "status": "success",
      "message": null
    }
  },
  {
    "business_description": "Creates multilingual voice-over AI solutions for educational publishers targeting global markets.",
    "domain_name_output": {
      "suggestions": [
        {"domain": "eduvoices.ai", "confidence": 0.92},
        {"domain": "multilangvoice.com", "confidence": 0.85},
        {"domain": "globaleduvox.io", "confidence": 0.8}
      ],
      "status": "success",
      "message": null
    }
  },
  {
    "business_description": "Develops cybersecurity tools for securing IoT devices in homes and small businesses.",
    "domain_name_output": {
      "suggestions": [
        {"domain": "homesafeiot.com", "confidence": 0.93},
        {"domain": "iotshield.pro", "confidence": 0.87},
        {"domain": "deviceguard.ai", "confidence": 0.81}
      ],
      "status": "success",
      "message": null
    }
  },
  {
    "business_description": "A B2C direct-to-consumer subscription delivering premium pet treats and toys monthly.",
    "domain_name_output": {
      "suggestions": [
        {"domain": "petjoybox.com", "confidence": 0.93},
        {"domain": "barktreatsclub.co", "confidence": 0.86},
        {"domain": "pawprimebox.shop", "confidence": 0.8}
      ],
      "status": "success",
      "message": null
    }
  },
  {
    "business_description": "A mobile app for tracking menstrual cycles, fertility windows, and women’s wellness with science-backed insights.",
    "domain_name_output": {
      "suggestions": [
        {"domain": "wellcycle.app", "confidence": 0.92},
        {"domain": "femtrackhealth.com", "confidence": 0.87},
        {"domain": "cycleinsights.io", "confidence": 0.8}
      ],
      "status": "success",
      "message": null
    }
  },
  {
    "business_description": "Builds plant-based vending machines stocked with fresh, healthy snacks for offices and universities.",
    "domain_name_output": {
      "suggestions": [
        {"domain": "plantvend.com", "confidence": 0.87},
        {"domain": "greenmachinestore.co", "confidence": 0.84},
        {"domain": "snackbotplant.shop", "confidence": 0.8}
      ],
      "status": "success",
      "message": null
    }
  },
  {
    "business_description": "An analytics platform for esports teams to track player stats, scrim results, and team performance trends.",
    "domain_name_output": {
      "suggestions": [
        {"domain": "esportsmetrics.com", "confidence": 0.9},
        {"domain": "gamestattrack.io", "confidence": 0.81},
        {"domain": "teamanalytix.app", "confidence": 0.8}
      ],
      "status": "success",
      "message": null
    }
  },
  {
    "business_description": "Creates career transition programs and digital resources for military veterans entering civilian tech roles.",
    "domain_name_output": {
      "suggestions": [
        {"domain": "veteranstechpath.com", "confidence": 0.91},
        {"domain": "at-easecareer.io", "confidence": 0.82},
        {"domain": "servicetotech.org", "confidence": 0.8}
      ],
      "status": "success",
      "message": null
    }
  },
  {
    "business_description": "A professional photography studio specializing in pet portraits, serving animal lovers in metropolitan areas.",
    "domain_name_output": {
      "suggestions": [
        {"domain": "urbanpetportraits.com", "confidence": 0.9},
        {"domain": "citypaws.studio", "confidence": 0.82},
        {"domain": "petphotomaster.co", "confidence": 0.8}
      ],
      "status": "success",
      "message": null
    }
  },
  {
    "business_description": "A monthly book subscription box, curated to support independent, diverse authors for avid readers.",
    "domain_name_output": {
      "suggestions": [
        {"domain": "authoredbox.com", "confidence": 0.91},
        {"domain": "readdiversitybox.co", "confidence": 0.85},
        {"domain": "bookblend.store", "confidence": 0.8}
      ],
      "status": "success",
      "message": null
    }
  },
  {
    "business_description": "A full-service 3D printing company for architects and real estate developers, from prototype to model delivery.",
    "domain_name_output": {
      "suggestions": [
        {"domain": "architectprint3d.com", "confidence": 0.89},
        {"domain": "protoprints.io", "confidence": 0.84},
        {"domain": "realtymodels.shop", "confidence": 0.8}
      ],
      "status": "success",
      "message": null
    }
  },
  {
    "business_description": "Offers climate-controlled, automated wine storage and personal cellaring services for wine collectors.",
    "domain_name_output": {
      "suggestions": [
        {"domain": "cellarvault.com", "confidence": 0.92},
        {"domain": "winekeeper.storage", "confidence": 0.87},
        {"domain": "storagemasterwine.co", "confidence": 0.8}
      ],
      "status": "success",
      "message": null
    }
  },
  {
    "business_description": "A platform for virtual author events and book launches, serving publishers and indie writers globally.",
    "domain_name_output": {
      "suggestions": [
        {"domain": "booklaunchlive.com", "confidence": 0.9},
        {"domain": "authormeetup.online", "confidence": 0.84},
        {"domain": "virtualreads.events", "confidence": 0.8}
      ],
      "status": "success",
      "message": null
    }
  },
  {
    "business_description": "A telehealth therapy service for rural youth offering access to licensed mental health professionals.",
    "domain_name_output": {
      "suggestions": [
        {"domain": "ruralteentherapy.com", "confidence": 0.89},
        {"domain": "youthmentalcare.health", "confidence": 0.81},
        {"domain": "younetwork.telehealth", "confidence": 0.78}
      ],
      "status": "success",
      "message": null
    }
  },
  {
    "business_description": "A SaaS application that allows supply chain managers to visualize global shipments in real-time and perform risk assessments.",
    "domain_name_output": {
      "suggestions": [
        {"domain": "chainsightpro.com", "confidence": 0.91},
        {"domain": "shipmentview.io", "confidence": 0.85},
        {"domain": "risksupplychain.app", "confidence": 0.8}
      ],
      "status": "success",
      "message": null
    }
  },
  {
    "business_description": "A specialized staffing agency matching remote software engineers with fintech startups.",
    "domain_name_output": {
      "suggestions": [
        {"domain": "fintechtalent.com", "confidence": 0.91},
        {"domain": "remotedevmatch.io", "confidence": 0.85},
        {"domain": "codeengineerstaffing.com", "confidence": 0.8}
      ],
      "status": "success",
      "message": null
    }
  },
  {
    "business_description": "An online course platform for personal finance education, including debt reduction and investment strategies.",
    "domain_name_output": {
      "suggestions": [
        {"domain": "wizardfinance.com", "confidence": 0.91},
        {"domain": "investwiseclass.online", "confidence": 0.85},
        {"domain": "debtkickstart.edu", "confidence": 0.8}
      ],
      "status": "success",
      "message": null
    }
  },
  {
    "business_description": "A micro-SaaS delivering simple, one-click GDPR privacy compliance tools for Shopify merchants.",
    "domain_name_output": {
      "suggestions": [
        {"domain": "gdprshoptools.com", "confidence": 0.89},
        {"domain": "privapps.app", "confidence": 0.84},
        {"domain": "datacomplynow.store", "confidence": 0.8}
      ],
      "status": "success",
      "message": null
    }
  },
  {
    "business_description": "An online community for programmers.",
    "domain_name_output": {
      "suggestions": [
        {"domain": "codersforum.com", "confidence": 0.78},
        {"domain": "devchatroom.net", "confidence": 0.69},
        {"domain": "programmerscircle.io", "confidence": 0.66}
      ],
      "status": "success",
      "message": null
    }
  },
  {
    "business_description": "LLM4ALL.",
    "domain_name_output": {
      "suggestions": [
        {"domain": "llm4all.com", "confidence": 0.55},
        {"domain": "llmforall.ai", "confidence": 0.53},
        {"domain": "4allllm.dev", "confidence": 0.50}
      ],
      "status": "success",
      "message": null
    }
  },
  {
    "business_description": "A service offering AI-based video editing for social media influencers and content creators throughout North America and Europe, emphasizing rapid turnaround times, brand-specific templates, multi-platform optimization, trend alert integrations, royalty-free audio options, and collaborative review features so creators can publish unique, standout content faster, while ensuring consistent, on-brand output and saving both time and money compared to traditional freelance editors or self-editing—additionally including onboarding guidance, content calendar suggestions, storage, and robust analytics.",
    "domain_name_output": {
      "suggestions": [
        {"domain": "editai.io", "confidence": 0.91},
        {"domain": "contentclip.app", "confidence": 0.88},
        {"domain": "fastbrandstudio.com", "confidence": 0.81}
      ],
      "status": "success",
      "message": null
    }
  },
  {
    "business_description": "Shop.",
    "domain_name_output": {
      "suggestions": [
        {"domain": "shoppilot.com", "confidence": 0.6},
        {"domain": "shopgear.store", "confidence": 0.57},
        {"domain": "shopnowly.co", "confidence": 0.54}
      ],
      "status": "success",
      "message": null
    }
  },
  {
    "business_description": "A global intelligence agency offering covert services.",
    "domain_name_output": {
      "suggestions": [],
      "status": "blocked",
      "message": "Sorry, we can't provide domain name suggestions for sensitive or potentially harmful business descriptions."
    }
  },
  {
    "business_description": "A social network for sharing pirated movies.",
    "domain_name_output": {
      "suggestions": [],
      "status": "blocked",
      "message": "This business description is not allowed due to potential copyright infringement."
    }
  },
  {
    "business_description": "Provides age-inappropriate dating services for minors.",
    "domain_name_output": {
      "suggestions": [],
      "status": "blocked",
      "message": "This business description contains unacceptable or harmful content."
    }
  },
  {
    "business_description": "A news platform promoting hate speech.",
    "domain_name_output": {
      "suggestions": [],
      "status": "blocked",
      "message": "Sorry, we cannot assist with harmful or inappropriate business descriptions."
    }
  },
  {
    "business_description": "???",
    "domain_name_output": {
      "suggestions": [],
      "status": "blocked",
      "message": "This business description is too ambiguous. Please clarify your idea."
    }
  },
  {
    "business_description": "",
    "domain_name_output": {
      "suggestions": [],
      "status": "blocked",
      "message": "The business description field is empty. Please provide more information."
    }
  },
  {
    "business_description": "DFS, B2B, proprietary ETL, SME, 360º CX.",
    "domain_name_output": {
      "suggestions": [],
      "status": "blocked",
      "message": "The business description uses excessive jargon or acronyms. Please clarify."
    }
  },
  {
    "business_description": "The business helps other businesses.",
    "domain_name_output": {
      "suggestions": [],
      "status": "blocked",
      "message": "This description is too vague. Please provide more details."
    }
  },
  {
    "business_description": "A platform for tracking real-time ocean freight shipments, including predictive ETAs, carbon impact analysis, route optimization, live map visualizations, integrations via RESTful APIs, granular multi-lane reporting, shipment-level chat, incident management with custom alert thresholds, temperature and humidity sensor support, blockchain audit trails, and carrier rating aggregations, for freight forwarders, global shippers, and logistics providers, focused on reducing supply chain risk and improving customer satisfaction across North America, Europe, and APAC.",
    "domain_name_output": {
      "suggestions": [
        {"domain": "shiptrackpro.com", "confidence": 0.91},
        {"domain": "freightvision.io", "confidence": 0.87},
        {"domain": "oceanoptimize.app", "confidence": 0.81}
      ],
      "status": "success",
      "message": null
    }
  }
]
"""

In [48]:
import json
finetuning_dataset = []
data_list = json.loads(businesses_result)
for i, item_dict in enumerate(data_list):
  validated_example = BusinessExample(**item_dict)
  business_description = validated_example.business_description
  domain_name_output_data = validated_example.domain_name_output.model_dump()
  output_json_string = json.dumps(domain_name_output_data, ensure_ascii=False)
  finetuning_dataset.append({
        "instruction": "Generate a list of unique, memorable, and relevant domain name suggestions for the given business description. Provide confidence scores for each suggestion.",
            "input": business_description,
            "output": output_json_string
        })

In [49]:
len(finetuning_dataset)

119

In [31]:
# import json
# finetuning_dataset = []
# for i, json_string_array_item in enumerate(distiset["default"]["train"]["generation"]):
#     try:
#         # 1. Parse the outer JSON string into a Python list
#         parsed_list = json.loads(json_string_array_item)
#         # 2. Check if the list is not empty and get the first (and only) dictionary
#         if not parsed_list:
#             raise ValueError("Parsed JSON list is empty.")

#         example_dict = parsed_list[0]
#         validated_example = BusinessExample(**example_dict)

#         business_description = validated_example.business_description
#         # Convert the nested DomainNameOutput Pydantic object to a dictionary for JSON output
#         domain_name_output_data = validated_example.domain_name_output.model_dump()
#         # 5. Convert the domain_name_output dictionary to a JSON string
#         output_json_string = json.dumps(domain_name_output_data, ensure_ascii=False)
#         finetuning_dataset.append({
#         "instruction": "Generate a list of unique, memorable, and relevant domain name suggestions for the given business description. Provide confidence scores for each suggestion.",
#             "input": business_description,
#             "output": output_json_string
#         })

#     except Exception as e:
#         print(f"Warning: Unexpected error processing row {i}: {e}. Skipping. JSON string: {json_string_array_item}\n{'-'*50}")


In [50]:
output_filename = "finetuning_data.jsonl"
with open(output_filename, "w") as f:
    for entry in finetuning_dataset:
        f.write(json.dumps(entry) + "\n")
print(f"\nFinetuning dataset saved to '{output_filename}'")
print(f"Total examples for finetuning: {len(finetuning_dataset)}")


Finetuning dataset saved to 'finetuning_data.jsonl'
Total examples for finetuning: 119
